# Question 8
***
JG Hanekom <br>
20780893 <br>
December <br>
***

## Setup

In [1]:
! apt update &> /dev/null
! apt install mongodb | tail
!service mongodb start
! pip install pymongo --quiet



invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Created symlink /etc/systemd/system/multi-user.target.wants/mongodb.service → /lib/systemd/system/mongodb.service.
Setting up mongo-tools (3.6.3-0ubuntu1) ...
Setting up mongodb (1:3.6.3-0ubuntu1.4) ...
Processing triggers for systemd (237-3ubuntu10.52) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

 * Starting database mongodb
   ...done.


In [2]:
! mongo --version

MongoDB shell version v3.6.3
git version: 9586e557d54ef70f9ca4b43c26892cd55257e1a5
OpenSSL version: OpenSSL 1.1.1  11 Sep 2018
allocator: tcmalloc
modules: none
build environment:
    distarch: x86_64
    target_arch: x86_64


## Libraries

In [3]:
import requests
import pprint
import pymongo
import json

## Question 1
> Produce code that reads from the events API and writes the responses as documents in a MongoDB
collection. [3]

In [4]:
# Request the data
url = 'https://us-ie-big-data-technologies.ew.r.appspot.com/api/v1/events?asset_contract_address=0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb&offset=0&limit=10000&occurred_after=2021-09-01T00%3A00%3A00%2B02%3A00&occurred_before=2021-10-01T00%3A00%3A00%2B02%3A00&collection_slug=cryptopunks'
headers = {"Accept": "application/json/"}
response = requests.request("GET", url, headers=headers)

# Display as json 
data = response.json()

# connect to localhost on default port 27017
client = pymongo.MongoClient('127.0.0.1', 27017)
db = client['cryptopunk_db']
collection_crypto = db['punk']

# Insert the json file to mongoDB
collection_crypto.insert_many(data["asset_events"])

In [5]:
# When nescessary to delete the database (fresh start)
# client.drop_database("cryptopunk_db")

## Question 2
>  How many event transactions are contained in MongoDB? Demonstrate by querying the database. [1]

In [6]:
collection_crypto.count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


534

## Question 3
>  Query the database and find the token identifier and selling price for the ten most expensive punks. [2]

In [7]:
# Define an empty query
myquery = {}

# Select total_price and token_id
select = {"total_price":1,
          "_id":0,
          "asset.token_id": 1}

# Submit query
result = collection_crypto.find(myquery,select).sort("total_price", -1).limit(10)

# Format and print Results
for i in result:
  # Convert from wei to ether
  i["total_price"] = float(i["total_price"][:-17])/10
  print('Token: {asset[token_id]} \t Price (ether): {total_price} '.format(**i))

Token: 3634 	 Price (ether): 99.9 
Token: 8922 	 Price (ether): 99.9 
Token: 7633 	 Price (ether): 99.9 
Token: 9103 	 Price (ether): 99.9 
Token: 1672 	 Price (ether): 99.7 
Token: 3239 	 Price (ether): 99.6 
Token: 3378 	 Price (ether): 99.5 
Token: 3533 	 Price (ether): 99.5 
Token: 2224 	 Price (ether): 99.0 
Token: 3173 	 Price (ether): 99.0 


## Question 4
> How many unique punks were sold in this period? [2]

In [8]:
result = collection_crypto.distinct("asset.token_id")
len(result)

395

## Question 5
>  On which day did the most punks sell (where "event_type": "successful") [2]

In [9]:
project_stage = {"$project": {
      "date": {
         "$dateFromString": {
            "dateString": '$created_date'
         }
      }
   }
}

group_stage = {"$group": {
      "_id": {
        "month": { "$month": "$date" },
        "day": { "$dayOfMonth": "$date" },
        "year": { "$year": "$date" }
      },
      "totalPrice": { "$sum": "$price" },
      "count": { "$sum": 1 }
    }
}

sort_stage = { "$sort" : { "count" : -1 } }

limit_stage = {"$limit": 10}

result = collection_crypto.aggregate( [project_stage, group_stage, sort_stage , limit_stage] )

print("-----------------------")
print("Date \t        Sold")
print("-----------------------")
for i in result:
  print('{_id[day]}/{_id[month]}/{_id[year]}: \t {count}'.format(**i))

-----------------------
Date 	        Sold
-----------------------
16/9/2021: 	 90
30/9/2021: 	 44
24/9/2021: 	 29
15/9/2021: 	 27
1/9/2021: 	 27
25/9/2021: 	 26
2/9/2021: 	 25
3/9/2021: 	 21
29/9/2021: 	 21
4/9/2021: 	 19


## Question 6
> What is the total value of Etherium traded on that day (using "event_type": successful)? [2]

I want to convert it to decimal but `$toDecimal` is not a recognised command. I see that I am running version 3.6.3 of MongoDB. One of the StackOverflow forums mentioned that `$toDecimal` and `$convert` can only be used in version 4.

In [10]:
project_stage = {"$project": {
      "date": {
         "$dateFromString": {
            "dateString": '$created_date'
         }
      }
   }
}

group_stage = {"$group": {
      "_id": {
        "month": { "$month": "$date" },
        "day": { "$dayOfMonth": "$date" },
        "year": { "$year": "$date" }
      },
      "totalPrice": { "$sum": "$total_price" },
      "count": { "$sum": 1 }
    }
}

sort_stage = { "$sort" : { "count" : -1 } }

limit_stage = {"$limit": 10}

result = collection_crypto.aggregate( [project_stage, group_stage, sort_stage , limit_stage] )

print("------------------------------------")
print("Date \t        Sold   Price (ether)")
print("------------------------------------")
for i in result:
  # Convert from wei to ether
  # i["totalPrice"] = float(i["totalPrice"][:-17])/10
  print('{_id[day]}/{_id[month]}/{_id[year]}: \t {count} \t    {totalPrice}'.format(**i))

------------------------------------
Date 	        Sold   Price (ether)
------------------------------------
16/9/2021: 	 90 	    0
30/9/2021: 	 44 	    0
24/9/2021: 	 29 	    0
15/9/2021: 	 27 	    0
1/9/2021: 	 27 	    0
25/9/2021: 	 26 	    0
2/9/2021: 	 25 	    0
3/9/2021: 	 21 	    0
29/9/2021: 	 21 	    0
4/9/2021: 	 19 	    0
